In [232]:
# Import Dependencies
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import pickle
import joblib
import json
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
number = preprocessing.LabelEncoder()

## Import and Check Chicago Crime Datasets

In [159]:
# Import 2016-2019 crime data
crime_2016 = os.path.join("..","Resources", "crime_clean_2016.csv") 
crime_2017 = os.path.join("..","Resources", "crime_clean_2017.csv")
crime_2018 = os.path.join("..","Resources", "crime_clean_2018.csv")
crime_2019 = os.path.join("..","Resources", "crime_clean_2019.csv")

crime_2016_df_final = pd.read_csv(crime_2016)
crime_2017_df_final = pd.read_csv(crime_2017)
crime_2018_df_final = pd.read_csv(crime_2018)

# 2019 is the test data
test_data = pd.read_csv(crime_2019)

# Join datasets for 2016, 2017, and 2018 for the training data
join1 = crime_2016_df_final.append(crime_2017_df_final)
training_data = join1.append(crime_2018_df_final)
X = training_data.append(test_data).copy()

In [119]:
#function to Convert data to numbers
from sklearn import preprocessing
def convert(data):
    number = preprocessing.LabelEncoder()
    data['date'] = number.fit_transform(data.date)
    data['time'] = number.fit_transform(data.time)
    data['block'] = number.fit_transform(data.block)
    data['description'] = number.fit_transform(data.description)
    data['location_description'] = number.fit_transform(data.location_description)
    data['iucr'] = number.fit_transform(data.iucr)
    data['fbi_code'] = number.fit_transform(data.fbi_code)
    data['primary_type'] = number.fit_transform(data.primary_type)
    data['domestic'] = number.fit_transform(data.domestic)
    data['latitude'] = number.fit_transform(data.latitude)
    data['longitude'] = number.fit_transform(data.longitude)
    data['arrest'] = number.fit_transform(data.arrest)
    return data

In [220]:
y

0         0
1         0
2         1
3         0
4         0
         ..
256903    0
256904    0
256905    0
256906    0
256907    0
Name: arrest, Length: 1044809, dtype: int64

In [244]:
c = X['location_description'].unique()
b = X['primary_type'].unique()

encoded_loc = number.fit_transform(c).astype('int')
encoded_type = number.fit_transform(b).astype('int')
#inverse_transform(encoded)
#c.shape
encoded_type

array([32,  2, 16,  6, 29, 33,  3,  5, 23,  1, 25,  9, 17, 18, 28,  8, 11,
       22, 13, 15, 30, 14, 20,  0, 31, 26,  4, 24,  7, 10, 12, 27, 19, 21])

In [197]:
b

array(['THEFT', 'BATTERY', 'LIQUOR LAW VIOLATION', 'CRIMINAL DAMAGE',
       'ROBBERY', 'WEAPONS VIOLATION', 'BURGLARY', 'CRIM SEXUAL ASSAULT',
       'OFFENSE INVOLVING CHILDREN', 'ASSAULT', 'OTHER OFFENSE',
       'DECEPTIVE PRACTICE', 'MOTOR VEHICLE THEFT', 'NARCOTICS',
       'PUBLIC PEACE VIOLATION', 'CRIMINAL TRESPASS', 'HOMICIDE',
       'OBSCENITY', 'INTERFERENCE WITH PUBLIC OFFICER', 'KIDNAPPING',
       'SEX OFFENSE', 'INTIMIDATION', 'NON-CRIMINAL', 'ARSON', 'STALKING',
       'PROSTITUTION', 'CONCEALED CARRY LICENSE VIOLATION',
       'OTHER NARCOTIC VIOLATION', 'CRIMINAL SEXUAL ASSAULT', 'GAMBLING',
       'HUMAN TRAFFICKING', 'PUBLIC INDECENCY', 'NON - CRIMINAL',
       'NON-CRIMINAL (SUBJECT SPECIFIED)'], dtype=object)

In [245]:
loc_dict = dict(zip(c,encoded_loc))
type_dict = dict(zip(b,encoded_type))

In [200]:
type_dict

{'THEFT': 32,
 'BATTERY': 2,
 'LIQUOR LAW VIOLATION': 16,
 'CRIMINAL DAMAGE': 6,
 'ROBBERY': 29,
 'WEAPONS VIOLATION': 33,
 'BURGLARY': 3,
 'CRIM SEXUAL ASSAULT': 5,
 'OFFENSE INVOLVING CHILDREN': 23,
 'ASSAULT': 1,
 'OTHER OFFENSE': 25,
 'DECEPTIVE PRACTICE': 9,
 'MOTOR VEHICLE THEFT': 17,
 'NARCOTICS': 18,
 'PUBLIC PEACE VIOLATION': 28,
 'CRIMINAL TRESPASS': 8,
 'HOMICIDE': 11,
 'OBSCENITY': 22,
 'INTERFERENCE WITH PUBLIC OFFICER': 13,
 'KIDNAPPING': 15,
 'SEX OFFENSE': 30,
 'INTIMIDATION': 14,
 'NON-CRIMINAL': 20,
 'ARSON': 0,
 'STALKING': 31,
 'PROSTITUTION': 26,
 'CONCEALED CARRY LICENSE VIOLATION': 4,
 'OTHER NARCOTIC VIOLATION': 24,
 'CRIMINAL SEXUAL ASSAULT': 7,
 'GAMBLING': 10,
 'HUMAN TRAFFICKING': 12,
 'PUBLIC INDECENCY': 27,
 'NON - CRIMINAL': 19,
 'NON-CRIMINAL (SUBJECT SPECIFIED)': 21}

In [181]:
loc_dict.get('HOUSE')

99

In [120]:
#function to drop unneeded columns/keeping only features needed for model
def set_data(data):
    data = (data[[
              #'date'
              #'day'
              'month'
              #,'year'
              #,'time'
              ,'hour'
              #,'month_day'
              ,'day_of_week'
              #,'district'
              #,'block'
              #,'ward'
              #,'beat'
              #,'community_area'
              #,'description'
              ,'location_description'
              #,'x_coordinate'
              #,'y_coordinate'
              #,'iucr'
              #,'fbi_code'
              ,'primary_type'
              #,'domestic'
              #,'latitude'
              #,'longitude'
            ]])
    return data

In [121]:
#LabelEncode the data
X = convert(X)
y = X["arrest"]

In [122]:
X = set_data(X)

In [134]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.1,test_size=0.1, random_state=42)

In [135]:
#Scale the data for SVM, requires normalization
#reference: https://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html
from sklearn.preprocessing import StandardScaler
y_train = np.array(y_train).reshape((len(y_train), 1),order='C')
y_test = np.array(y_test).reshape((len(y_test), 1),order='C')

X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [136]:
#model is faster with C-contiguous data
#reference: https://scikit-learn.org/stable/modules/svm.html
X_train_scaled = np.asarray(X_train_scaled, order='C')
X_test_scaled = np.asarray(X_test_scaled, order='C')

In [126]:
X_train_scaled.shape

(208961, 5)

In [127]:
y_train.ravel().shape

(208961,)

In [128]:
y_train

array([[0],
       [1],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int64)

In [129]:
y_test.shape

(208962, 1)

In [131]:
# Support vector machine linear classifier
from sklearn.neural_network import MLPClassifier


model = MLPClassifier( max_iter=7000,activation='relu',alpha=.001,hidden_layer_sizes=(50, 100, 50),
                     learning_rate='invscaling', solver= 'adam',random_state=2)

model.fit(X_train_scaled, y_train.ravel())

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(50, 100, 50), learning_rate='invscaling',
              learning_rate_init=0.001, max_iter=7000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=2, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [137]:
model = MLPClassifier( max_iter=500,activation='tanh',alpha=.05,hidden_layer_sizes=(50, 100, 50),
                     learning_rate='adaptive', solver= 'adam')
model.fit(X_train_scaled, y_train.ravel())

MLPClassifier(activation='tanh', alpha=0.05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(50, 100, 50), learning_rate='adaptive',
              learning_rate_init=0.001, max_iter=500, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [138]:
#https://gist.github.com/WittmannF/60680723ed8dd0cb993051a7448f7805
def accuracy(confusion_matrix):
   diagonal_sum = confusion_matrix.trace()
   sum_of_all_elements = confusion_matrix.sum()
   return diagonal_sum / sum_of_all_elements

#Predicting y for X_val
y_pred = model.predict(X_test_scaled)

#Importing Confusion Matrix
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
#Comparing the predictions against the actual observations in y_val
cm = confusion_matrix(y_pred, y_test)

#Printing the accuracy
print('Accuracy of MLPClassifier :', accuracy(cm))
print('Test Acc: %.3f' % model.score(X_test_scaled, y_test))
print(classification_report(y_test, y_pred))

Accuracy of MLPClassifier : 0.8651429446502235
Test Acc: 0.865
              precision    recall  f1-score   support

           0       0.87      0.98      0.92     83112
           1       0.85      0.41      0.56     21369

    accuracy                           0.87    104481
   macro avg       0.86      0.70      0.74    104481
weighted avg       0.86      0.87      0.85    104481



In [133]:
# save the model to disk
filename = 'MLP_model.sav'
joblib.dump(model, filename)

['MLP_model.sav']

In [143]:
X.head(1)

,month,hour,day_of_week,location_description,primary_type
0,12,23,5,3,32


In [144]:
training_data.head(10)

,id,date,day,month,year,time,hour,month_day,day_of_week,district,...,location_description,x_coordinate,y_coordinate,iucr,fbi_code,primary_type,domestic,latitude,longitude,arrest
0,10819224,2016-12-31T23:59:00.000,31,12,2016,23:59:00.000,23,1231,5,16,...,AIRPORT BUILDING NON-TERMINAL - SECURE AREA,1100658,1934241,0810,06,THEFT,False,41.976290,-87.905227,False
1,10801137,2016-12-31T23:58:00.000,31,12,2016,23:58:00.000,23,1231,5,5,...,RESIDENCE,1178014,1829709,0430,04B,BATTERY,False,41.688033,-87.623931,False
2,10801110,2016-12-31T23:55:00.000,31,12,2016,23:55:00.000,23,1231,5,19,...,RESIDENCE,1166154,1920300,2250,22,LIQUOR LAW VIOLATION,False,41.936885,-87.664770,True
3,10802006,2016-12-31T23:55:00.000,31,12,2016,23:55:00.000,23,1231,5,1,...,HOTEL/MOTEL,1176964,1902140,0486,08B,BATTERY,True,41.886815,-87.625593,False
4,10801865,2016-12-31T23:54:00.000,31,12,2016,23:54:00.000,23,1231,5,6,...,RESIDENCE,1178949,1853139,1310,14,CRIMINAL DAMAGE,True,41.752307,-87.619798,False
5,10802735,2016-12-31T23:50:00.000,31,12,2016,23:50:00.000,23,1231,5,17,...,ALLEY,1147645,1927783,031A,03,ROBBERY,False,41.957795,-87.732601,False
6,10801291,2016-12-31T23:50:00.000,31,12,2016,23:50:00.000,23,1231,5,14,...,RESIDENCE,1152529,1911998,0460,08B,BATTERY,False,41.914384,-87.715064,False
7,10801171,2016-12-31T23:47:00.000,31,12,2016,23:47:00.000,23,1231,5,1,...,HOTEL/MOTEL,1177683,1902638,0486,08B,BATTERY,True,41.888165,-87.622937,False
8,10801143,2016-12-31T23:45:00.000,31,12,2016,23:45:00.000,23,1231,5,15,...,STREET,1144064,1904203,0334,03,ROBBERY,True,41.893157,-87.746359,False
9,10803002,2016-12-31T23:45:00.000,31,12,2016,23:45:00.000,23,1231,5,19,...,SIDEWALK,1165426,1921325,0320,03,ROBBERY,False,41.939713,-87.667416,False


In [202]:
primary_type

32

In [249]:
month = 6
hour = 10
day_of_week = 4
location_description = loc_dict.get('AIRPORT BUILDING NON-TERMINAL - SECURE AREA')
primary_type =type_dict.get('BURGLARY')
        
filename='RF_model.sav'
model = joblib.load(filename)
print(model.predict([[month, hour, day_of_week,location_description,primary_type]]))

[0]


In [221]:
#Printing the accuracy
print('Accuracy of MLPClassifier :', accuracy(cm))
print('Test Acc: %.3f' % model.score(X_test_scaled, y_test))
y_pred = model.predict(X_test_scaled)
print(classification_report(y_test, y_pred))

Accuracy of MLPClassifier : 0.8651429446502235
Test Acc: 0.874
              precision    recall  f1-score   support

           0       0.88      0.98      0.92     83112
           1       0.84      0.48      0.61     21369

    accuracy                           0.87    104481
   macro avg       0.86      0.73      0.77    104481
weighted avg       0.87      0.87      0.86    104481



In [247]:
file = os.path.join("..","Resources", "loc_dict.txt") 
f = open(file,"w")
f.write(str(loc_dict))
f.close()

In [248]:
file = os.path.join("..","Resources", "type_dict.txt") 
f = open(file,"w")
f.write(str(type_dict))
f.close()

In [222]:
X_test

,month,hour,day_of_week,location_description,primary_type
129660,7,18,2,132,32
70972,9,13,1,69,32
109138,7,12,0,17,9
151727,6,7,2,132,3
186079,4,22,3,124,8
...,...,...,...,...,...
93751,8,22,5,157,32
58597,10,12,4,154,32
171551,5,9,0,124,32
250085,1,10,1,15,18
